In [34]:
%reload_ext autoreload
%autoreload 2
import sys
from dotenv import load_dotenv
import os
from sklearn.metrics import log_loss
load_dotenv()

ROOT = os.getenv("ROOT")
sys.path.append(ROOT)
assert ROOT is not None, "Please create a .env file and specify your ROOT path."
from src.processing import DataProcessor

[12:02:24] | INFO  | loaded 701528 rows                                                            ]8;id=799092;file:///Users/lucanyckees/Desktop/my-repos/recommendation_system/src/processing.py\processing.py]8;;\:]8;id=168364;file:///Users/lucanyckees/Desktop/my-repos/recommendation_system/src/processing.py#33\33]8;;\

In [17]:
y_pred_proba = xgb.predict_proba(X_test_tfidf)
log_loss_value = log_loss(y_test, y_pred_proba)
print(f"Log Loss (Cross Entropy): {log_loss_value}")

Log Loss (Cross Entropy): 0.39912544479695994
